# Publication "Practitioner’s Guide to Statistical Tests" (VK Team)

In [1]:
import pandas as pd
import numpy as np
from random import randint
from scipy import stats 
from typing import Tuple

In [2]:
def generate_data(skew: float = 2.0,
                  N: int = 5000,
                  NN: int = 2000,
                  success_rate: float = 0.02,
                  uplift: float = 0.1,
                  beta: float = 250.) -> Tuple[
    Tuple[np.ndarray, np.ndarray], Tuple[np.ndarray, np.ndarray], np.ndarray]:
    """
    Generates experimental data for N users in NN experiments
    :param skew: float, skewness of views distribution
    :param N: int, number of users in each experimental group (in control and in treatment)
    :param NN: int, number of experiments
    :param success_rate: float, mean success rate in control group
    :param uplift: float, relative uplift of mean success rate in treatment group
    :param beta: float, parameter of success rate distribution
    :return: (np.array, np.array, np.array, np.array, np.array) shape (NN, N), views in control group,
    clicks in control group, views in treatment group, clicks in treatment group, ground truth user CTRs for control group
    """
    views_0 = np.exp(stats.norm(1, skew).rvs(NN * N)).astype(int).reshape(NN, N) + 1
    views_1 = np.exp(stats.norm(1, skew).rvs(NN * N)).astype(int).reshape(NN, N) + 1

    # views are always positive, abs is fixing numerical issues with high skewness
    views_0 = np.absolute(views_0)
    views_1 = np.absolute(views_1)

    alpha_0 = success_rate * beta / (1 - success_rate)
    success_rate_0 = stats.beta(alpha_0, beta).rvs(NN * N).reshape(NN, N)

    alpha_1 = success_rate * (1 + uplift) * beta / (1 - success_rate * (1 + uplift))
    success_rate_1 = stats.beta(alpha_1, beta).rvs(NN * N).reshape(NN, N)

    clicks_0 = stats.binom(n=views_0, p=success_rate_0).rvs()
    clicks_1 = stats.binom(n=views_1, p=success_rate_1).rvs()
    return ((views_0.astype(np.float64), clicks_0.astype(np.float64)),
            (views_1.astype(np.float64), clicks_1.astype(np.float64)),
            success_rate_0.astype(np.float64))

In [3]:
success_rate = 0.05
uplift = 0.2
N = 5
NN = 3

beta = 1000
skew = 1

ab_params = {'success_rate': success_rate, 'uplift': uplift, 'beta': beta, 'skew': skew, 'N': N, 'NN': NN}

In [4]:
(view_control, click_control), (view_treatment, click_treatment), gt_success_rates = generate_data(**ab_params)

In [5]:
control = pd.DataFrame(data={'click':list(click_control),'view':list(view_control)})
treatment = pd.DataFrame({'click':list(click_treatment),'view':list(view_treatment)})

In [6]:
control.head()

,click,view
0,"[0.0, 0.0, 1.0, 0.0, 0.0]","[7.0, 1.0, 7.0, 2.0, 5.0]"
1,"[0.0, 0.0, 0.0, 0.0, 1.0]","[3.0, 3.0, 2.0, 4.0, 1.0]"
2,"[0.0, 0.0, 0.0, 0.0, 0.0]","[2.0, 20.0, 2.0, 1.0, 3.0]"


In [7]:
treatment.head()

,click,view
0,"[0.0, 0.0, 0.0, 0.0, 2.0]","[3.0, 2.0, 1.0, 2.0, 19.0]"
1,"[0.0, 0.0, 0.0, 0.0, 0.0]","[6.0, 4.0, 3.0, 1.0, 4.0]"
2,"[0.0, 0.0, 0.0, 0.0, 0.0]","[3.0, 1.0, 2.0, 2.0, 1.0]"


In [8]:
def linearization_of_clicks(clicks_0, views_0, clicks_1, views_1):
    """
    Fits linear model clicks = k * views and returns clicks - k * views (e.g. it accounts for correlation of
    clicks and views)
    :param clicks_0: np.array shape (n_experiments, n_users), clicks of every user from control group in every experiment
    :param views_0: np.array shape (n_experiments, n_users), views of every user from control group in every experiment
    :param clicks_1: np.array shape (n_experiments, n_users), clicks of every user from treatment group in every experiment
    :param views_1: np.array shape (n_experiments, n_users), views of every user from treatment group in every experiment
    :return: (np.array, np_array) shape (n_experiments), linearized clicks for every user in every experiment
    """
    k = (clicks_0.sum(axis=1) / views_0.sum(axis=1)).reshape(-1, 1)
    L_0 = clicks_0 - k * views_0
    L_1 = clicks_1 - k * views_1
    return L_0, L_1

In [9]:
linearized_control, linearized_treatment = linearization_of_clicks(click_control, 
                                                                   view_control, 
                                                                   click_treatment, 
                                                                   view_treatment)

In [10]:
control['linearized_click'] = list(linearized_control)
treatment['linearited_click'] = list(linearized_treatment)

In [11]:
control.head(100)

,click,view,linearized_click
0,"[0.0, 0.0, 1.0, 0.0, 0.0]","[7.0, 1.0, 7.0, 2.0, 5.0]","[-0.3181818181818182, -0.045454545454545456, 0..."
1,"[0.0, 0.0, 0.0, 0.0, 1.0]","[3.0, 3.0, 2.0, 4.0, 1.0]","[-0.23076923076923078, -0.23076923076923078, -..."
2,"[0.0, 0.0, 0.0, 0.0, 0.0]","[2.0, 20.0, 2.0, 1.0, 3.0]","[0.0, 0.0, 0.0, 0.0, 0.0]"


In [12]:
treatment.head()

,click,view,linearited_click
0,"[0.0, 0.0, 0.0, 0.0, 2.0]","[3.0, 2.0, 1.0, 2.0, 19.0]","[-0.13636363636363635, -0.09090909090909091, -..."
1,"[0.0, 0.0, 0.0, 0.0, 0.0]","[6.0, 4.0, 3.0, 1.0, 4.0]","[-0.46153846153846156, -0.3076923076923077, -0..."
2,"[0.0, 0.0, 0.0, 0.0, 0.0]","[3.0, 1.0, 2.0, 2.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0]"


In [13]:
def t_test(a, b):
    """
    Calculates two-sided t-test p-values for multiple experiments
    :param a: np.array shape (n_experiments, n_users), metric values in control group
    :param b: np.array shape (n_experiments, n_users), metric values in treatment group
    :return: np.array shape (n_experiments), two-sided p-values of t-test in all experimetns
    """
    result = list(map(lambda x: stats.ttest_ind(x[0], x[1]).pvalue, zip(a, b)))
    return np.array(result)

In [14]:
t_test(linearized_control, linearized_treatment)

array([0.6238804 , 0.28245358,        nan])